In [178]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [179]:
# Get cpu or gpu device for training.
device = "cpu" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [180]:
# reading csv files
data =  pd.read_csv('data\zoo\zoo.data', sep=",", header=None)


In [181]:
y = data[17]
names = data[0]
X = data.loc[:, 1:16]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# X = torch.from_numpy(X.values).float().to(device)
# y = torch.torch.from_numpy(y.values).long().to(device)
# y = y-1


X_train = torch.from_numpy(X_train).float().to(device)
y_train = torch.torch.from_numpy(y_train.values).long().to(device) -1
X_test = torch.from_numpy(X_test).float().to(device)
y_test = torch.torch.from_numpy(y_test.values).long().to(device) -1


In [182]:
# make network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.linear_relu_stack = nn.Sequential(
        nn.Linear(16, 300), # input layer
        nn.ReLU(), # activation function
        nn.Dropout(),
        nn.Linear(300, 150), # hidden layer
        nn.ReLU(), # activation function
        nn.Dropout(),
        nn.Linear(150, 7), # output layer

    )

    def forward(self, x):
        out = self.linear_relu_stack(x)
        return out


In [183]:
def training_loop(n_epochs, optimizer, model, loss_fn, X, y):

    total = len(y_train)

    for epoch in range(1, n_epochs+1):
        correct = 0
        outputs = model(X)
        loss = loss_fn(outputs, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, dim=1) # Gives us the index of the highest value

        correct += int((predicted == y).sum())
        acc = correct/total

        # epoch_list.append(epoch)
        # train_loss_list.append(loss)

        # set when to print info about training progress
        if epoch == 1 or epoch % 10 == 0:
            print('Epoch {}, Training loss {}, Accuracy {:.2f}'.format(epoch, np.sqrt(loss.to('cpu').detach().numpy()), acc))

n_epochs = 20
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()




In [184]:
def validate_on_test(model, loss_fn, X_test, y_test):
    total = len(y_test)
    correct = 0
    with torch.no_grad():
        outputs = model(X_test)
        loss_test = loss_fn(outputs, y_test)

        _, predicted = torch.max(outputs, dim=1) # Gives us the index of the highest value
        correct += int((predicted == y_test).sum())


    print('Test Loss: {}, Accuracy {:.4f}'.format(loss_test, correct/total))

In [185]:
training_loop(n_epochs=n_epochs,
              optimizer=optimizer,
              model=model,
              loss_fn=loss_fn,
              X=X_train,
              y=y_train)

validate_on_test(model=model, loss_fn=loss_fn, X_test=X_test, y_test=y_test)

Epoch 1, Training loss 1.3984730243682861, Accuracy 0.21
Epoch 10, Training loss 0.3281100392341614, Accuracy 0.95
Epoch 20, Training loss 0.12258364260196686, Accuracy 1.00
Test Loss: 0.08637832850217819, Accuracy 0.9615
